In [1]:
# move extracted game's assets to lr and hr folder
import os
from os.path import join
import shutil
root_folder = 'R:/ev'
hr_folder = 'E:/Dataset/hamidashi/hr'
lr_folder = 'E:/Dataset/hamidashi/lr'

for folder_path in [lr_folder, hr_folder]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# Use os.walk() to traverse through all directories and subdirectories
for root, dirs, files in os.walk(root_folder):
    for file in files:
        if "png" in file and file[0] == 'z':
            name = file[1:]
            if name not in files:
                continue
            shutil.move(join(root, file), join(hr_folder, name))
            shutil.move(join(root, name), join(lr_folder, name))
